In [35]:
%%writefile mapper1.py

import sys
import re

for line in sys.stdin:
    try:
        article_id, content = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    words = re.split('\W*\s+\W*', content,flags=re.UNICODE)
    for word in words:
        sys.stderr.write(f'reporter:counter:Wiki stats,Total words,1\n')
        if ((word[1:].islower()) and (word[0].isupper()) and (not word[0].isdigit())):
            print(f'{word.lower()}\t1\t1')
        else:
            print(f'{word.lower()}\t0\t1')


Overwriting mapper1.py


In [36]:
%%writefile reducer1.py

import sys

current_key = None
total_amt = 0
total_tc = 0
for line in sys.stdin:
    try:
        key, count, total_count = line.strip().split('\t', 2)
        count = int(count)
        total_count = int(total_count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print(f'{current_key}\t{total_amt}\t{total_tc}')
        total_amt = 0
        total_tc = 0
        current_key = key
    total_amt += count
    total_tc += total_count

if current_key:
    print(f'{current_key}\t{total_amt}\t{total_tc}')



Overwriting reducer1.py


In [39]:
%%writefile mapper2.py

import sys

for line in sys.stdin:
    try:
        word, amt , t_amt = line.strip().split('\t',2)
        amt = int(amt)
        t_amt = int(t_amt)

    except ValueError as e:
        continue
        
        
    if float(amt)/float(t_amt) > 0.995:
        print(f'{amt}\t{word}')
            
    


Overwriting mapper2.py


In [38]:
%%writefile reducer2.py

import sys

for line in sys.stdin:
    try:
        amt, word = line.strip().split('\t',1)
        amt = int(amt)
    except ValueError as e:
        continue
    
    print(f'{word}\t{amt}')


Overwriting reducer2.py


In [30]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")
HADOOP_JAR="/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar"

hdfs dfs -rm -r -skipTrash assignment* > /dev/null

# Code for your first job
yarn jar ${HADOOP_JAR} \
    -D mapreduce.job.name="First Job" \
    -D mapreduce.job.reduces=4 \
    -files mapper1.py,reducer1.py \
    -mapper "python3 mapper1.py" \
    -combiner "python3 reducer1.py" \
    -reducer "python3 reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR2="final_result"

hdfs dfs -rm -r -skipTrash final_result* > /dev/null

# Code for your second job
yarn jar ${HADOOP_JAR} \
    -D mapreduce.job.name="Second Job" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options="-k1,1nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python3 mapper2.py" \
    -reducer "python3 reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null



hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n '5p'



rm: `assignment*': No such file or directory
20/09/19 13:58:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/19 13:58:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/09/19 13:58:16 ERROR streaming.StreamJob: Error Launching job : Permission denied: user=jovyan, access=EXECUTE, inode="/tmp/hadoop-yarn":root:supergroup:drwxrwx---
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:350)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkTraverse(FSPermissionChecker.java:311)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:238)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:189)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkTraverse(FSPermissionChecker.java:541)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkTraverse(FSDirectory.java